# Python登录微信
需要在终端运行以下语句：   
```bash
pip install itchat
pip install echarts-python
pip install jieba
pip install pillow
pip install wordcloud
pip install matplotlib
pip install numpy
```

## 统计好友性别比例

In [1]:
# coding:utf-8
import itchat
'''
微信好友性别比例
'''

# 先登录
itchat.login()

# 获取好友列表
friends = itchat.get_friends(update=True)[0:]

# 初始化计数器，有男有女，当然，有些人是不填的
male = female = other = 0

# 遍历这个列表，列表里第一位是自己，所以从"自己"之后开始计算
# 1表示男性，2女性
for i in friends[1:]:
    sex = i["Sex"]
    if sex == 1:
        male += 1
    elif sex == 2:
        female += 1
    else:
        other += 1

# 总数算上，好计算比例啊～
total = len(friends[1:])

# 打印结果
print(u"男性好友：%.2f%%" % (float(male) / total * 100))
print(u"女性好友：%.2f%%" % (float(female) / total * 100))
print(u"未填性别：%.2f%%" % (float(other) / total * 100))



Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 陈艺荣


男性好友：59.63%
女性好友：33.35%
未填性别：7.01%


## 获取好友信息

In [2]:
friends=itchat.get_friends(update=True)[0:]

## 生成好友个性签名词云

In [3]:
import re
tList = []
for i in friends:
    signature = i["Signature"].replace(" ", "").replace("span", "").replace("class", "").replace("emoji", "")
    rep = re.compile("1f\d.+")
    signature = rep.sub("", signature)
    tList.append(signature)

# 拼接字符串
text = "".join(tList)

# jieba分词
import jieba
wordlist_jieba = jieba.cut(text, cut_all=True)
wl_space_split = " ".join(wordlist_jieba)

# wordcloud词云
import matplotlib.pyplot as plt
from wordcloud import WordCloud, ImageColorGenerator
import os
import numpy as np
import PIL.Image as Image


d = os.path.dirname("./")
# 更改目录下Wordcloud生成图片，如：xiaohuangren.jpg
alice_coloring = np.array(Image.open(os.path.join("./", "xiaohuangren.jpg")))
# win系统需要更改font路径，如：C:\Windows\Fonts\msyhbd.ttc
my_wordcloud = WordCloud(background_color="white", max_words=2000, mask=alice_coloring,
                         max_font_size=40, random_state=42,
                         font_path='C:\\Windows\\Fonts\\ARIALUNI.ttf')\
    .generate(wl_space_split)

image_colors = ImageColorGenerator(alice_coloring)
plt.imshow(my_wordcloud.recolor(color_func=image_colors))
plt.imshow(my_wordcloud)
plt.axis("off")
plt.show()

# 保存图片 并发送到手机
my_wordcloud.to_file(os.path.join(d, "wechat_cloud.png"))
itchat.send_image("wechat_cloud.png", 'filehelper')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\TF\AppData\Local\Temp\jieba.cache
Loading model cost 0.627 seconds.
Prefix dict has been built succesfully.


<Figure size 640x480 with 1 Axes>

<ItchatReturnValue: {'BaseResponse': {'Ret': 0, 'ErrMsg': '请求成功', 'RawMsg': '请求成功'}, 'MsgID': '1313775741784964652', 'LocalID': '15616002151960'}>

## 头像拼接

In [ ]:
import itchat
import os
import math
import PIL.Image as Image
import matplotlib.pyplot as plt
import random
from wordcloud import WordCloud
import jieba
import re
# 获取头像
def headImg(friends):
    #itchat.login()
    #friends = itchat.get_friends(update=True)
    #itchat.get_head_img() 获取到头像二进制，并写入文件，保存每张头像
    for count, f in enumerate(friends):
        # 根据userName获取头像
        img = itchat.get_head_img(userName=f["UserName"])
        imgFile = open("img/" + str(count) + ".jpg", "wb")
        imgFile.write(img)
        imgFile.close()


# 头像拼接图
def createImg():
    x = 0
    y = 0
    imgs = os.listdir("img")
    random.shuffle(imgs)
    # 创建640*640的图片用于填充各小图片
    newImg = Image.new('RGBA', (640, 640))
    # 以640*640来拼接图片，math.sqrt()开平方根计算每张小图片的宽高，
    width = int(math.sqrt(640 * 640 / len(imgs)))
    # 每行图片数
    numLine = int(640 / width)

    for i in imgs:
        try:
            img = Image.open("img/" + i)
        # 缩小图片
            img = img.resize((width, width), Image.ANTIALIAS)
        # 拼接图片，一行排满，换行拼接
            newImg.paste(img, (x * width, y * width))
            x += 1
            if x >= numLine:
                x = 0
                y += 1
        except IOError:
            print("img/ %s can not open"%(i))
    newImg.save("all.png")
    
# 微信好友头像拼接
headImg(friends)
createImg()

## 自动回复好友信息

In [ ]:
import time
# 自动回复
# 封装好的装饰器，当接收到的消息是Text，即文字消息
@itchat.msg_register('Text')
def text_reply(msg):
    # 当消息不是由自己发出的时候
    if not msg['FromUserName'] == myUserName:
        # 发送一条提示给文件助手
        itchat.send_msg(u"[%s]收到好友@%s 的信息：%s\n" %
                        (time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(msg['CreateTime'])),
                         msg['User']['NickName'],
                         msg['Text']), 'filehelper')
        # 回复给好友
        return u'[自动回复]您好，我现在有事不在，一会再和您联系。\n已经收到您的的信息：%s\n' % (msg['Text'])

if __name__ == '__main__':
    itchat.auto_login()

    # 获取自己的UserName
    myUserName = itchat.get_friends(update=True)[0]["UserName"]
    itchat.run()